In [1]:
import torch
import torchvision
import urllib
from PIL import Image
import os
os.chdir("../")
from util import masking

In [2]:
# Load alexnet
alexnet = torchvision.models.alexnet(weights='DEFAULT')

In [3]:
# Load image file from pytorch
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "img/dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [4]:
# Process image
input_image = Image.open(filename)
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    alexnet.to('cuda')

# Evaluate model
with torch.no_grad():
    output = alexnet(input_batch)

# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities.shape)

torch.Size([1000])


In [5]:
# Use the GetActivationsHook to get the activations after layer 9
hooked_alexnet = masking.HookedModel(alexnet)
mask_hook = masking.GetActivationsHook('layer9')
hooked_alexnet.apply_hook('features.9',mask_hook)
with torch.no_grad():
    alexnet(input_batch)
hooked_alexnet.remove_hooks()

In [6]:
# The GetActivationsHook conveniently provides a get activations function
mask_hook.get_activations().shape

torch.Size([1, 256, 13, 13])

In [7]:
# Use the OutputMaskHook to mask the activations after layer 9 using the size
mask_hook = masking.OutputMaskHook(torch.randn([1, 256, 13, 13]))
hooked_alexnet.apply_hook('features.9',mask_hook)
with torch.no_grad():
    outputs_masked = alexnet(input_batch)
probabilities_masked = torch.nn.functional.softmax(outputs_masked[0], dim=0)

In [8]:
torch.equal(probabilities,probabilities_masked)

False